# Genetic attribution preprocessing

In this notebook we overview the [Genetic Engineering Attrinution competition in Driven Data](https://www.drivendata.org/competitions/63/genetic-engineering-attribution/).   

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from collections import OrderedDict

## Loading data

In [2]:
%%time
parent_dir = "/home/rio/genetic_engineering_attribution"
data_dir = os.path.join(parent_dir,"data")
data_list = os.listdir(data_dir)
data_list

CPU times: user 543 µs, sys: 70 µs, total: 613 µs
Wall time: 338 µs


['train_labels.csv',
 'train_values.csv',
 'test_values.csv',
 'submission_format_3TFRxH6.csv']

#### Training features

In [3]:
%%time
train_features_path = os.path.join(data_dir,data_list[1])
train_features = pd.read_csv(train_features_path)
print("shape of train_features: ", train_features.shape)
train_features.head()

shape of train_features:  (63017, 41)
CPU times: user 2.17 s, sys: 168 ms, total: 2.33 s
Wall time: 2.33 s


,sequence_id,sequence,bacterial_resistance_ampicillin,bacterial_resistance_chloramphenicol,bacterial_resistance_kanamycin,bacterial_resistance_other,bacterial_resistance_spectinomycin,copy_number_high_copy,copy_number_low_copy,copy_number_unknown,...,species_budding_yeast,species_fly,species_human,species_mouse,species_mustard_weed,species_nematode,species_other,species_rat,species_synthetic,species_zebrafish
0,9ZIMC,CATGCATTAGTTATTAATAGTAATCAATTACGGGGTCATTAGTTCA...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5SAQC,GCTGGATGGTTTGGGACATGTGCAGCCCCGTCTCTGTATGGAGTGA...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,E7QRO,NNCCGGGCTGTAGCTACACAGGGCGGAGATGAGAGCCCTACGAAAG...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CT5FP,GCGGAGATGAAGAGCCCTACGAAAGCTGAGCCTGCGACTCCCGCAG...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7PTD8,CGCGCATTACTTCACATGGTCCTCAAGGGTAACATGAAAGTGATCC...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Training targets

In [4]:
%%time
train_targets_path = os.path.join(data_dir,data_list[0])
train_targets = pd.read_csv(train_targets_path)
print("shape of train_targets: ", train_targets.shape)
train_targets.head()

shape of train_targets:  (63017, 1315)
CPU times: user 6.64 s, sys: 387 ms, total: 7.03 s
Wall time: 7.03 s


,sequence_id,00Q4V31T,012VT4JK,028IO5W2,03GRNN7N,03Y3W51H,09MQV1TY,0A4AHRCT,0A9M05NC,0B9GCUVV,...,ZQNGGY33,ZSHS4VJZ,ZT1IP3T6,ZU6860XU,ZU6TVFFU,ZU75P59K,ZUI6TDWV,ZWFD8OHC,ZX06ZDZN,ZZJVE4HO
0,9ZIMC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5SAQC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,E7QRO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CT5FP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7PTD8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Preprocessing

#### One-hot encoding DNA sequences

In [5]:
def dna2dict(dna_seq):
    """
    DNA bases are ordered in tuples alphabetically as (A,C,G,N,T)
    """
    dna_dict = OrderedDict()
    dna_seq = dna_seq.upper()
    for ix,b in enumerate(dna_seq):
        dna_dict[ix] = (int(b=="A"),int(b=="C"),int(b=="G"),int(b=="N"),int(b=="T"))
    return dna_dict
        

In [6]:
dna_dict = dna2dict(train_features.sequence.values[1])
dna_dict

OrderedDict([(0, (0, 0, 1, 0, 0)),
             (1, (0, 1, 0, 0, 0)),
             (2, (0, 0, 0, 0, 1)),
             (3, (0, 0, 1, 0, 0)),
             (4, (0, 0, 1, 0, 0)),
             (5, (1, 0, 0, 0, 0)),
             (6, (0, 0, 0, 0, 1)),
             (7, (0, 0, 1, 0, 0)),
             (8, (0, 0, 1, 0, 0)),
             (9, (0, 0, 0, 0, 1)),
             (10, (0, 0, 0, 0, 1)),
             (11, (0, 0, 0, 0, 1)),
             (12, (0, 0, 1, 0, 0)),
             (13, (0, 0, 1, 0, 0)),
             (14, (0, 0, 1, 0, 0)),
             (15, (1, 0, 0, 0, 0)),
             (16, (0, 1, 0, 0, 0)),
             (17, (1, 0, 0, 0, 0)),
             (18, (0, 0, 0, 0, 1)),
             (19, (0, 0, 1, 0, 0)),
             (20, (0, 0, 0, 0, 1)),
             (21, (0, 0, 1, 0, 0)),
             (22, (0, 1, 0, 0, 0)),
             (23, (1, 0, 0, 0, 0)),
             (24, (0, 0, 1, 0, 0)),
             (25, (0, 1, 0, 0, 0)),
             (26, (0, 1, 0, 0, 0)),
             (27, (0, 1, 0, 0, 0)),
  

In [7]:
len(dna_dict)

456

In [8]:
list(dna_dict.values())

[(0, 0, 1, 0, 0),
 (0, 1, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (0, 0, 1, 0, 0),
 (1, 0, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (0, 0, 1, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 0, 0, 1),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (0, 0, 1, 0, 0),
 (0, 0, 1, 0, 0),
 (1, 0, 0, 0, 0),
 (0, 1, 0, 0, 0),
 (1, 0, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (0, 1, 0, 0, 0),
 (1, 0, 0, 0, 0),
 (0, 0, 1, 0, 0),
 (0, 1, 0, 0, 0),
 (0, 1, 0, 0, 0),
 (0, 1, 0, 0, 0),
 (0, 1, 0, 0, 0),
 (0, 0, 1, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 1, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 1, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (0, 0, 0, 0, 1),
 (1, 0, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (0, 0, 1, 0, 0),
 (1, 0, 0, 0, 0),
 (0, 0, 1, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (1, 0, 0, 0, 0),
 (0, 1, 0, 0, 0),
 (1, 0, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (1, 0, 0, 0, 0),
 (0, 0, 1, 0, 0),
 (1, 0, 0, 0, 0),
 (0, 0, 1, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (0, 0, 0,

#### One-hot encoded DNA sequences as dataframes

In [9]:
def dna2df(dna_seq,seq_id=None, lab_id = None):
    dna_dict = dna2dict(dna_seq)
    dna_df = pd.DataFrame(dna_dict.values(),columns=("A","C","G","N","T"))
    if seq_id:
        columns = ["sequence_id"] + list(dna_df.columns)
        dna_df["sequence_id"] = [seq_id]*len(dna_df)
        dna_df = dna_df.loc[:,columns]
    if lab_id:
        columns = ["lab_id"] + list(dna_df.columns)
        dna_df["lab_id"] = [lab_id]*len(dna_df)
        dna_df = dna_df.loc[:,columns]
    return dna_df


In [10]:
dna_seq = train_features.sequence.values[0]
seq_id = train_features.sequence_id.values[0]
lab_id = train_targets.columns[train_targets.iloc[0,:]==1][0]
dna_df = dna2df(dna_seq=dna_seq,seq_id=seq_id,lab_id=lab_id)
dna_df

,lab_id,sequence_id,A,C,G,N,T
0,RYUA3GVO,9ZIMC,0,1,0,0,0
1,RYUA3GVO,9ZIMC,1,0,0,0,0
2,RYUA3GVO,9ZIMC,0,0,0,0,1
3,RYUA3GVO,9ZIMC,0,0,1,0,0
4,RYUA3GVO,9ZIMC,0,1,0,0,0
...,...,...,...,...,...,...,...
7146,RYUA3GVO,9ZIMC,1,0,0,0,0
7147,RYUA3GVO,9ZIMC,0,1,0,0,0
7148,RYUA3GVO,9ZIMC,0,1,0,0,0
7149,RYUA3GVO,9ZIMC,0,0,1,0,0


In [11]:
lab_id

'RYUA3GVO'

#### Generating separate csvs for each lab

In [12]:
%%time
split_labs_dir = os.path.join(parent_dir,"split_labs")
if not os.path.isdir(split_labs_dir):
    os.mkdir(split_labs_dir)
#unique_labs = train_targets.columns[1:]
#pbar = tqdm(unique_labs)
#for lab in tqdm(unique_labs):
#    lab_path = os.path.join(split_labs_dir,lab)
#    if not os.path.isdir(lab_path):
#        pbar.set_description(f"Generating directory {lab_path}")
#        os.mkdir(lab_path)
#    else:
#        pbar.set_description(f"Directory {lab_path} already exists. Skipping.")


CPU times: user 238 µs, sys: 16 µs, total: 254 µs
Wall time: 268 µs


In [13]:
%%time
split_labs_dir = os.path.join(parent_dir,"split_labs")
def split_labs(train_features,train_targets,savedir=split_labs_dir,usecols=None):
    pbar = tqdm(train_features.iterrows(),total=len(train_features))
    if usecols is None:
        usecols = list(train_features.columns[train_features.columns.get_loc("sequence")+1:])
    unique_labs = list(train_targets.columns[1:])
    pbar = tqdm(unique_labs)
    #print("unique_labs; ", unique_labs)
    for lab_id in pbar:
        #print("lab_id: ", lab_id)
        pbar.set_description(f"Processing lab {lab_id}")
        ixs = train_targets.index[train_targets.loc[:,lab_id]==1]
        df_lab_concat = []
        for ix in ixs:
            row = train_features.loc[ix,:]
            dna_seq = row.sequence
            seq_id = row.sequence_id
            lab_id = train_targets.columns[train_targets.iloc[ix,:]==1][0]
            dna_df = dna2df(dna_seq,seq_id,lab_id)
        #pbar.set_description(f"Processing sample {seq_id}")
        #print("dna_df: ", dna_df)
            if usecols:
                repeat = row.loc[usecols].values.reshape(1,-1)
                repeat = np.repeat( repeat,len(dna_df), axis=0)
                #print("shape: ", np.repeat(sample.loc[usecols].values,len(dna_df),axis=1).shape)
                #print(np.repeat( sample.loc[usecols].values,len(dna_df)  ))
                extra_cols_df = pd.DataFrame( repeat ,columns=usecols)   
                dna_df = pd.concat([dna_df,extra_cols_df],axis=1)
            df_lab_concat.append(dna_df)
        df_lab = pd.concat(df_lab_concat,axis=0,ignore_index=True)
        if savedir is not None:
            savepath = os.path.join(savedir,lab_id+".csv")
            #print("savepath: ", savepath)
            df_lab.to_csv(savepath,index=True)
    
    
    

#### Saving a .csv for each lab

In [ ]:
%%time
split_labs(train_features,train_targets,split_labs_dir,usecols=None)

  0%|          | 0/63017 [00:00<?, ?it/s][A

Processing lab 1KZHNVYR:   4%|▍         | 52/1314 [02:49<43:06,  2.05s/it]


Processing lab 2VTLZHDS:   8%|▊         | 103/1314 [06:10<33:16,  1.65s/it]


Processing lab 40MD0YZ3:  12%|█▏        | 154/1314 [10:03<1:28:01,  4.55s/it]


Processing lab 5AUVXXDU:  16%|█▌        | 205/1314 [12:05<1:27:30,  4.73s/it]


Processing lab 6UGWNYCX:  20%|█▉        | 257/1314 [14:14<1:26:08,  4.89s/it]


...

Processing lab C35C2C2W:  35%|███▌      | 462/1314 [24:44<35:32,  2.50s/it]


Processing lab DLSU0QRX:  39%|███▉      | 513/1314 [25:38<13:00,  1.03it/s]


Processing lab F50DBVIK:  43%|████▎     | 565/1314 [27:05<18:04,  1.45s/it]


Processing lab GUCIE6TT:  47%|████▋     | 617/1314 [31:27<3:31:13, 18.18s/it]


Processing lab I7FXTVDP:  51%|█████     | 666/1314 [33:01<23:17,  2.16s/it]

In [ ]:
%%time
for i in train_features.iterrows():
    print(i)

In [ ]:
train_features

In [ ]:
train_features.loc[:,"sequence"]